# Test 22 part 2

Energy computations and various visualizations

In [2]:
import os
import sys

import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import clear_output

import matplotlib as mpl 
#mpl.use('pgf')
import matplotlib.pyplot as plt

sys.path.insert(0, '../icenumerics/')
import icenumerics as ice

import auxiliary as aux
import montecarlo_tools as mc
import chirality_tools as chir
ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

In [3]:
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# Parameters

In [4]:
quench_time = 300*ureg.s
evolution_time = 60*ureg.s
total_time = quench_time + evolution_time
data_path = "../data/test23/"
runs = 10

params = {
    "particle_radius":1.4*ureg.um,
    "particle_susceptibility":0.4,
    "particle_diffusion":0.14*ureg.um**2/ureg.s,
    "particle_temperature":300*ureg.K,
    "particle_density":1000*ureg.kg/ureg.m**3,

    "trap_sep":3*ureg.um,
    "trap_height":8*ureg.pN*ureg.nm,
    "trap_stiffness":100e-3*ureg.pN/ureg.nm,
    "height_spread":0,
    "susceptibility_spread":0,
    "isperiodic":True,

    "total_time":total_time,
    "framespersec":20*ureg.Hz,
    "dt":0.1*ureg.ms,
    "max_field":10*ureg.mT,
    "sim_temp":300*ureg.K,
    "sim_dipole_cutoff":40*ureg.um,
}

params["lattice_constant"] = params["trap_sep"]+(2*params["particle_radius"]+1*ureg.um)*np.sqrt(2)

params['mu0'] = (4*np.pi)*1e-7 * ureg.H/ureg.m
params['m'] = np.pi * (2*params['particle_radius'])**3 *params['particle_susceptibility']*params['max_field']/6/params['mu0']
params['kb'] = 1.380649e-23 * ureg.J / ureg.K
params['kbT'] = (params['kb'] * params['sim_temp']).to(ureg.nm * ureg.pN)

# Constructing AF2 states

Here the goal is to make several AF2 states for many sizes

In [88]:
def save_af2(N):
    a = params["lattice_constant"]
    sp = ice.spins()
    sp.create_lattice("square",[N,N],lattice_constant=a, border="periodic")

    particle = ice.particle(radius = params["particle_radius"],
                susceptibility = params["particle_susceptibility"],
                diffusion = params["particle_diffusion"],
                temperature = params["particle_temperature"],
                density = params["particle_density"])

    trap = ice.trap(trap_sep = params["trap_sep"],
                height = params["trap_height"],
                stiffness = params["trap_stiffness"])

    col = ice.colloidal_ice(sp, particle, trap,
                            height_spread = params["height_spread"], 
                            susceptibility_spread = params["susceptibility_spread"],
                            periodic = params["isperiodic"])

            
    col.region = np.array([[0,0,-3*(params["particle_radius"]/a/N).magnitude],[1,1,3*(params["particle_radius"]/a/N).magnitude]])*N*a
        
    params['particle'] = particle
    params['trap'] = trap

    col1 = col.copy(deep=True)
    
    pps = int(N**2)
    
    flipsv = [pps + k + n for k in range(0,pps,N) for n in range(0,N,2)]
    flipsh = [0 + k + n for k in range(0,pps,N) for n in range(1,N,2)]
    flipsh2 = [0 + k + n for k in range(0,pps,N*2) for n in range(0,N,1)]
    flips = flipsv + flipsh + flipsh2
    col1 = mc.flip_colloids(col1, indices=flips)

    col1.to_ctrj().to_csv(os.path.join(data_path,'af2',f'{N}.csv'))


In [ ]:
for size in tqdm(range(10,31)):
    save_af2(size)

# AF4

I will prepare all AF4 states from the AF2 with a number of flips

In [ ]:

for size in tqdm(range(10,31)):
    
    # Importing the file
    ctrj = pd.read_csv(os.path.join(data_path,'af2',f'{size}.csv'), index_col=0)
    
    # Declaring some variables
    particle = ice.particle(
        radius = params['particle_radius'],
        susceptibility = params['particle_susceptibility'],
        diffusion = params["particle_diffusion"],
        temperature = params["particle_temperature"],
        density = params["particle_density"]
    )
    
    trap = ice.trap(
        trap_sep = params["trap_sep"], 
        height = params["trap_height"],
        stiffness = params["trap_stiffness"]
    )
    
    params['particle'] = particle
    params['trap'] = trap

    col = aux.get_colloids_from_ctrj2(ctrj,params)
    
    col1 = col.copy(deep=True)
    
    pps = int(size**2)
    flips = [i+j for i in range(pps+size,2*pps,2*size) for j in range(size)]
    col1 = mc.flip_colloids(col1,indices=flips)
    
    col1.to_ctrj().to_csv(os.path.join(data_path,'af4',f'{size}.csv'))
    

## Sanity check

In [ ]:
N = 11
params['size'] = N
ctrj = pd.read_csv(os.path.join(data_path,'af4',f'{N}.csv'), index_col= 0)
col = aux.get_colloids_from_ctrj2(ctrj,params)

v = ice.vertices()
v = v.colloids_to_vertices(col)

fig, ax = plt.subplots(figsize=(10,10))
col.display(ax)
v.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)
plt.show()


# Disordered state

Now I want to compute the energy of all obtained disordered states

In [128]:
def transform_energy(E,E4):
    return 1+ E/np.abs(E4)

In [104]:
def get_energy_at_realization(params,data_path,size,realization):
    
    params['size'] = size
    
    trj = pd.read_csv(os.path.join(data_path,str(size),'trj',f'trj{realization}.csv'), index_col=[0,1])
    last_frame = trj.index.get_level_values('frame').unique()[-1]
    particles = aux.get_coordinates_at_frame(trj,last_frame)
    dis_energy = aux.calculate_energy(params,particles)
    return dis_energy
    


In [ ]:
sim_path = '../data/test22'

energies = []
for size in tqdm(range(10,20,1)):
    cure = [get_energy_at_realization(params,sim_path,size,i) for i in range(1,5+1)]
    energies.append(cure)
    clear_output(wait=True)
energies

In [114]:
headers = [f'r{i}' for i in range(1,5+1)]
df = pd.DataFrame(data=energies, columns=headers)
df['size'] = list(range(10,20))
df.to_csv(os.path.join(data_path,'energies.csv'))

In [ ]:
gses = []
for size in tqdm(range(10,20)):
    params['size'] = size
    ctrj_sel = pd.read_csv(os.path.join(data_path,'af4',f'{size}.csv'), index_col= 0)
    gse = aux.calculate_energy(params, aux.get_positions_from_ctrj(ctrj_sel) )
    gses.append(gse)

gse

In [169]:
df['gs'] = gses
df.to_csv(os.path.join(data_path,'energies.csv'))

In [172]:
z = []

for i,row in df.iterrows():
   zsel = [transform_energy(row[f'r{k}'], row['gs']) for k in range(1,5+1) ]
   z.append(zsel)


In [ ]:
dfz = pd.DataFrame(data=z,columns=headers)
dfz.to_csv(os.path.join(data_path,'zs.csv'))
dfz

In [180]:
sizes = list(range(10,20))
z_mean = dfz.mean(axis=1).to_numpy()
z_std = dfz.std(axis=1).to_numpy()

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
#ax.scatter(sizes,z_mean)
ax.set_ylim((0,0.1))
ax.set_xlim((9,20))
ax.errorbar(sizes,z_mean,fmt='o',yerr=z_std,barsabove=True)


ax.set_xlabel('$n$')
ax.set_ylabel('$z$')


plt.show()

## Some other energy visualizations

In [42]:
def transformation(E,E4,N,kbT):
    return (E - E4)/(N*kbT)

In [ ]:
energies = pd.read_csv(os.path.join(data_path,'energies.csv'), index_col= 0)
energies

Getting the AF2 energy

In [ ]:
af2s = []
for size in tqdm(range(10,20)):
    params['size'] = size
    ctrj_sel = pd.read_csv(os.path.join(data_path,'af2',f'{size}.csv'), index_col= 0)
    gse = aux.calculate_energy(params, aux.get_positions_from_ctrj(ctrj_sel) )
    af2s.append(gse)

af2s

In [64]:
eta = []

for i,row in energies.iterrows():
   zsel = [transformation(row[f'r{k}'], row['gs'], row['size']**2,params['kbT'].magnitude) for k in range(1,5+1) ]
   eta.append(zsel)


In [ ]:
af2cul = [transformation(e,gs,n**2,params['kbT'].magnitude) for n,gs,e, in zip(energies['size'],energies['gs'],af2s)]
af2cul

In [ ]:
etadf = pd.DataFrame(data = eta, columns = [f'r{i}' for i in range(1,6)])
etadf

In [66]:
sizes = list(range(10,20))
eta_mean = etadf.mean(axis = 1).to_list()
eta_std = etadf.std(axis = 1).to_list()

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
#ax.scatter(sizes,z_mean)
ax.set_ylim((0,120))
ax.set_xlim((9,20))
ax.errorbar(sizes,eta_mean,fmt='o',yerr=eta_std,barsabove=True)

ax.scatter(sizes,af2cul, c='red')

ax.legend(['AF2','Disoredered'], loc='lower right')

ax.set_xlabel('$n$')
ax.set_ylabel('$(E - E_{GS})/(Nk_bT)$')

plt.show()

# Frame visualizations

In [21]:
vis_path = '../data/test22'

In [22]:
params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,10,realization=1)

In [7]:
N = 10
params['size'] = N
ctrj = pd.read_csv(os.path.join(vis_path,str(N),'ctrj','ctrj1.csv'),index_col=[0,1])
vrt = pd.read_csv(os.path.join(vis_path,str(N),'vertices','vertices1.csv'),index_col=[0,1])
last_frame = vrt.index.get_level_values('frame').unique()[-1]


v = ice.vertices()
v.vertices = vrt

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(20,20))

ax = axes[0][0]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,10,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)


ax = axes[0][1]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,15,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)

ax = axes[1][0]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,20,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)


ax = axes[1][1]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,30,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)

plt.show()

In [37]:
fig.savefig(os.path.join(data_path,'foursizes.png'),dpi=300,bbox_inches='tight')
fig.savefig(os.path.join(data_path,'foursizes.pdf'),bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(20,20))

ax = axes[0][0]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,10,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display_afgroup(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)


ax = axes[0][1]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,15,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display_afgroup(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)

ax = axes[1][0]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,20,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display_afgroup(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)


ax = axes[1][1]

params,ctrj,v,last_frame = aux.get_ctrj_and_vertices_from_file(params,vis_path,30,realization=1)
region_limit = params['size']*params['lattice_constant'].magnitude
N = params['size']
ax.set_title(f'$n={N}$',fontsize=20)

ice.draw_frame((aux.dropvis(ctrj)), frame_no=last_frame,
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)
v.display_afgroup(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)

plt.show()

In [39]:
fig.savefig(os.path.join(data_path,'foursizes_group.png'),dpi=300,bbox_inches='tight')
fig.savefig(os.path.join(data_path,'foursizes_group.pdf'),bbox_inches='tight')